![Crowded city](images/city-1265055_1280.jpg)

In the quest for efficiency and effectiveness in urban transportation, finding the optimal routes to take passengers from their initial locations to their desired destinations is paramount. This challenge is not just about reducing travel time; it's about enhancing the overall experience for both drivers and passengers, ensuring safety, and minimizing environmental impact. 

You have been asked to revolutionize the way taxis navigate the urban landscape, ensuring passengers reach their destinations swiftly, safely, and satisfactorily. As an initial step, your goal is to build a reinforcement learning agent that solves this problem within a simulated environment.

## The Taxi-v3 environment
The Taxi-v3 environment is a strategic simulation, offering a grid-based arena where a taxi navigates to address daily challenges akin to those faced by a taxi driver. This environment is defined by a 5x5 grid where the taxi's mission involves picking up a passenger from one of four specific locations (marked as Red, Green, Yellow, and Blue) and dropping them off at another designated spot. The goal is to accomplish this with minimal time on the road to maximize rewards, emphasizing the need for route optimization and efficient decision-making for passenger pickup and dropoff.

### Key Components:
- **Action Space:** Comprises six actions where 0 moves the taxi south, 1 north, 2 east, 3 west, 4 picks up a passenger, and 5 drops off a passenger.
- **Observation Space:** Comprises 500 discrete states, accounting for 25 taxi positions, 5 potential passenger locations, and 4 destinations. 
- **Rewards System:** Includes a penalty of -1 for each step taken without other rewards, +20 for successful passenger delivery, and -10 for illegal pickup or dropoff actions. Actions resulting in no operation, like hitting a wall, also incur a time step penalty.

![Taxi-v3 environment snapshot](images/Taxi_snap.png)


In [6]:
# Re-run this cell to install and import the necessary libraries and load the required variables
!pip install gymnasium[toy_text] imageio



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:

import numpy as np
import gymnasium as gym
import imageio
from IPython.display import Image
from gymnasium.utils import seeding

# Initialize the Taxi-v3 environment
env = gym.make("Taxi-v3", render_mode='rgb_array')

# Seed the environment for reproducibility
env.np_random, _ = seeding.np_random(42)
env.action_space.seed(42)
np.random.seed(42)

# Maximum number of actions per training episode
max_actions = 100 

In [8]:
# Q-learning parameters
alpha = 0.1  # Learning rate
gamma = 0.6  # Discount factor
epsilon = 0.1  # Exploration rate

# Initialize Q-table
q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(f'Q-table shape: {q_table.shape}')

# Training parameters
num_episodes = 10000

# Training the agent
for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    
    for _ in range(max_actions):
        if np.random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[state])  # Exploit
        
        next_state, reward, done, _, _ = env.step(action)
        
        # Q-learning update
        old_value = q_table[state, action]
        # print(f'Old value: {old_value}')
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        # print(f'q_table[{state}, {action}]: {q_table[state, action]}')
        
        state = next_state
        
        if done:
            break

print("Training completed.")

# Function to evaluate the agent
def evaluate_agent(num_episodes=100):
    total_rewards = 0
    for _ in range(num_episodes):
        state, _ = env.reset()
        done = False
        episode_reward = 0
        
        for _ in range(max_actions):
            action = np.argmax(q_table[state])
            state, reward, done, _, _ = env.step(action)
            episode_reward += reward
            if done:
                break
        
        total_rewards += episode_reward
    
    average_reward = total_rewards / num_episodes
    return average_reward

# Evaluate the trained agent
average_reward = evaluate_agent()
print(f"Average reward over 100 episodes: {average_reward}")

# Visualize a single episode
# def visualize_episode():
#     state, _ = env.reset()
#     done = False
#     frames = []
    
#     for _ in range(max_actions):
#         frame = env.render()
#         frames.append(frame)
        
#         action = np.argmax(q_table[state])
#         state, _, done, _, _ = env.step(action)
        
#         if done:
#             break
    
#     imageio.mimsave('taxi_episode.gif', frames, duration=0.5)
#     return Image(filename='taxi_episode.gif')

# # Display the visualization
# visualize_episode()



Q-table shape: (500, 6)
Training completed.
Average reward over 100 episodes: -3.86


In [10]:
# Once you are done, run this cell to visualize the agent's behavior through the episode
# Save frames as a GIF
imageio.mimsave('taxi_agent_behavior.gif', frames, fps=5)

# Display GIF
gif_path = "taxi_agent_behavior.gif" 
Image(gif_path) 

NameError: name 'frames' is not defined